In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# First let's kick off with summary of findings from previosu notebooks: 

- There are several cities which could be grouped together based on ride patterns and trends as well as demographics
- Some cities display higher variation in their ride distibutions which could be attributed to seasonality
- Due to this we could decide to either use DiD as out impact measurement framework which leverages data from before campaign launch well and accounts for pre existing differences between the cities
- MMM could also be a valid choice due to the nature of Bolt as a company as it considers other marketing channels and external factors 
- Some supporting metrics such as ROI, Customer Acquisition Cost or iCPA are also a must to analyze to understand the bigger picture and effecitveness of the campaign
- The Snapchat campaign seems to have made little difference so the 5000 Euro budget might have been too little or the medium/ campaign content/ campaign location might have not been the most effective

# Now to come up with budget calculations I will need to make a few assumptions based on the task data: 

Minimum Cost Per Acquisition (ride) in euros = 6  
Maximum Cost Per Acquisition (ride) in euros = 12  
Minimum Cost Per Thousand Impressions in euros = 2  
Maximum Cost Per Thousand Impressions in euros = 4  
Average profit per ride in euros = 6  
Last month's Snapchat budget in euros = 5000  ( this is not an assumption as it was said in the task)

And very importantly we need to assume:

- Snapchat campaign impact rate - 15% (new rides vs last month attributed to the Snapchat campaign)
- How much we want to improve on the Snapchat campaign - 1.2x
- CPA we are aiming for - 0.95 Snapchat's CPA

We also must calculate what would be a statstically significant effect as the task asks 'What minimal budget should the campaign have, so that we are able to measure its impact afterwards?' .

In [6]:
import pandas as pd
import numpy as np

# data
df = pd.read_csv(r'C:\Users\kaczm\Desktop\Bolt Marketing\Test Task - Marketing Measurement - DF.csv')
df['date'] = pd.to_datetime(df['date'])

CPA_MIN = 6  
CPA_MAX = 12  
CPM_MIN = 2  
CPM_MAX = 4  
PROFIT_PER_RIDE = 6  
SNAPCHAT_BUDGET = 5000  # as in the task

cities = [col for col in df.columns if col != 'date']
num_cities = len(cities)

august_2023 = df[df['date'].dt.to_period('M') == '2023-08']
july_2023 = df[df['date'].dt.to_period('M') == '2023-07']
august_2022 = df[df['date'].dt.to_period('M') == '2022-08']

august_2023_rides = august_2023[cities].sum().sum()
july_2023_rides = july_2023[cities].sum().sum()
august_2022_rides = august_2022[cities].sum().sum()

# let's calc ride increase
month_over_month_increase = august_2023_rides - july_2023_rides
year_over_year_increase = august_2023_rides - august_2022_rides

# the rate as assummed in the markdown cell
SNAPCHAT_IMPACT_RATE = 0.15 
NEW_RIDES = 4730
SNAPCHAT_NEW_RIDES = NEW_RIDES * SNAPCHAT_IMPACT_RATE
SNAPCHAT_CPA = SNAPCHAT_BUDGET / SNAPCHAT_NEW_RIDES

print(f"Snapchat Campaign Analysis (August 2023):")
print(f"Total rides in August 2023: {august_2023_rides:.0f}")
print(f"Total rides in July 2023: {july_2023_rides:.0f}")
print(f"Total rides in August 2022: {august_2022_rides:.0f}")
print(f"Month-over-month increase: {month_over_month_increase:.0f}")
print(f"Year-over-year increase: {year_over_year_increase:.0f}")
print(f"\nEstimated Snapchat impact:")
print(f"Estimated new rides from Snapchat: {SNAPCHAT_NEW_RIDES:.0f}")
print(f"Estimated Snapchat CPA: €{SNAPCHAT_CPA:.2f}")

# potentially attutable rides
if month_over_month_increase > 0:
    snapchat_contribution_mom = (SNAPCHAT_NEW_RIDES / month_over_month_increase) * 100
    print(f"\nPotential Snapchat contribution to month-over-month increase: {snapchat_contribution_mom:.2f}%")
else:
    print("\nRide numbers decreased compared to last month, making it difficult to attribute impact to Snapchat.")

if year_over_year_increase > 0:
    snapchat_contribution_yoy = (SNAPCHAT_NEW_RIDES / year_over_year_increase) * 100
    print(f"Potential Snapchat contribution to year-over-year increase: {snapchat_contribution_yoy:.2f}%")
else:
    print("Ride numbers decreased compared to last year, making it difficult to attribute impact to Snapchat.")



Snapchat Campaign Analysis (August 2023):
Total rides in August 2023: 200413
Total rides in July 2023: 195683
Total rides in August 2022: 196514
Month-over-month increase: 4730
Year-over-year increase: 3899

Estimated Snapchat impact:
Estimated new rides from Snapchat: 710
Estimated Snapchat CPA: €7.05

Potential Snapchat contribution to month-over-month increase: 15.00%
Potential Snapchat contribution to year-over-year increase: 18.20%


In [7]:
import pandas as pd
import numpy as np
from scipy import stats

last_month = df[df['date'] >= df['date'].max() - pd.Timedelta(days=30)]
avg_daily_rides_per_city = last_month[cities].mean().mean()
total_monthly_rides = avg_daily_rides_per_city * 30 * num_cities

# we need minimum detectable effect based on the number of days, alpha and power
def min_detectable_effect(n, alpha=0.05, power=0.8):
    t_alpha = stats.t.ppf(1 - alpha/2, df=2*n-2)
    t_beta = stats.t.ppf(power, df=2*n-2)
    min_effect = (t_alpha + t_beta) * np.sqrt(2/n)
    return min_effect

baseline_std = df[cities].std().mean()

# min budget for detectability
def calculate_minimal_budget_for_detectability():
    target_effect_size = 0.2  # aim to detect a 20% increase
    n_days = 30  # 30 days campaign - one month
    
    min_effect = min_detectable_effect(n_days)
    required_change = target_effect_size * baseline_std
    
    if required_change < min_effect * baseline_std:
        required_change = min_effect * baseline_std
    
    expected_new_rides = required_change * n_days
    minimal_budget = expected_new_rides * SNAPCHAT_CPA  
    
    return max(minimal_budget, SNAPCHAT_BUDGET)

minimal_budget = calculate_minimal_budget_for_detectability()
print(f"\nRecommended minimal budget for detectability: €{minimal_budget:.2f}")


expected_new_rides = minimal_budget / SNAPCHAT_CPA
min_effect = min_detectable_effect(30)  
min_detectable_change = min_effect * baseline_std * 30

print(f"Minimum detectable change in monthly rides: {min_detectable_change:.0f}")
print(f"This represents a {(min_detectable_change / total_monthly_rides) * 100:.2f}% change in total monthly rides")

# no of new rides
estimated_new_rides = minimal_budget / SNAPCHAT_CPA
print(f"Estimated new rides from TweetX: {estimated_new_rides:.0f}")

# potentail ROI
roi = ((estimated_new_rides * PROFIT_PER_RIDE) - minimal_budget) / minimal_budget * 100
print(f"Potential ROI: {roi:.2f}%")

budget_per_city = minimal_budget / num_cities
print(f"Number of cities that could be targeted: {num_cities}")
print(f"Budget per city: €{budget_per_city:.2f}")

# Ccompare vs 
impact_percentage = (estimated_new_rides / total_monthly_rides) * 100
print(f"This budget could potentially increase monthly rides by approximately {impact_percentage:.2f}%")


Recommended minimal budget for detectability: €22592.24
Minimum detectable change in monthly rides: 3206
This represents a 1.81% change in total monthly rides
Estimated new rides from TweetX: 3206
Potential ROI: -14.86%
Number of cities that could be targeted: 13
Budget per city: €1737.86
This budget could potentially increase monthly rides by approximately 1.81%
